In [1]:
"""Program Name: GALAXYMASS
Program that calls upon a function which takes in two parameters: Filename and and particle type.
    3 types: Halo type (1), Disk type (2), Bulge (3) 


Program returns the TOTAL MASS of any galaxy component 
"""

In [2]:
"""Import Packages"""
import numpy as np
import astropy.units as u
import astropy.table as tbl
from Readfile import Read


In [39]:
def ComponentMass(filename, particle_type):
    #print(filename, particle_type) #Test to verify proper file picked and p_type chosen
    
    #Read data file in: MW_000, M31_000, M33_000
    time, P_total, data = Read(filename)
    
    
    #Identity component(particle type)
    index = np.where(data["type"]==particle_type)
    #print (index)
    
    #Select mass values
    m = data['m'][index]
    #print (m)
    
    m_total = np.round(np.sum(m)*1e10*u.M_sun,3)
    #print (m_total)
    return m_total

In [177]:
# Stored names of files to iterate through to compute component mass values
fnames = ['MW_000', 'M31_000', 'M33_000', "Local Group"]

# Empty array used to store computed values from the For Loop for making Table
Data_tbl = np.zeros(0)

# Values of total mass for each component to be stored and called upon later in loop
TotalHalo = 0.0
TotalDisk = 0.0
TotalBulge = 0.0

# Start of For Loop
for file in fnames: #loop through fnames and calculate galaxy components mass
    
    # First check in fnames list iterated through
    if file != "Local Group":
        
        # Strip unneeded characters from data file name and store Galaxy Name
        GalaxyName = file.strip("_000") 
   
        # Calculate component Masses for particle types, 1-3
        HaloMass = np.round(ComponentMass(file,1)/1e12,3)
        DiskMass = np.round(ComponentMass(file,2)/1e12,3)
        BulgeMass = np.round(ComponentMass(file,3)/1e12,3)

        # Calculate Total Mass values for each component
        TotalHalo += HaloMass
        TotalDisk += DiskMass
        TotalBulge += BulgeMass
        TotalMass = HaloMass+DiskMass+BulgeMass

        # Determine Baryon fraction (total stellar mass, total mass)
        f_bar = np.round((BulgeMass+DiskMass)/TotalMass,3)


        # store variables into list
        Eval = [GalaxyName,HaloMass, DiskMass, BulgeMass, TotalMass, f_bar]
        

        # A loop to append each calculated value to empty Data_tbl array
        for j in Eval:
            Data_tbl = np.append(Data_tbl, j)
            
    #Do this if failed first check
    else:
        
        # Else statement to Calculates Local Group Values
        # Not input file for ComponentMass function, takes in Calculated sums define before start of Loop
        GalaxyName = file
        
        # Calculated the total Local Group Mass
        TotalLocal = TotalHalo+TotalDisk+TotalBulge
        
        # Calculated the Baryon Fraction for Local Group
        f_local = (TotalDisk+TotalBulge)/TotalLocal
        
        #List of Local Group values to appended to Data_tbl array 
        Local = [GalaxyName, np.round(TotalHalo,3), np.round(TotalDisk,3), np.round(TotalBulge,3), np.round(TotalLocal,3), np.round(f_local,3)]
        
        # Loop that appends each values in Local list to Data_tbl
        for k in Local:
            
            # Append the kth index in Local to Data_tbl
            Data_tbl = np.append(Data_tbl,k)


# Reshape array to 4 by 6 for table arguments.
Data_array = np.reshape(Data_tbl,[4,6])

"""Table Properties"""

# Define each column name argument
Col_names = ['Galaxy', 'Halo Mass $(10^{12}M_{\odot})$', 'Disk Mass $(10^{12}M_{\odot})$', 'Bulge Mass $(10^{12}M_{\odot})$', 'Total Mass $(10^{12}M_{\odot})$', '$f_{bar}$']

# Make table store as t
t = tbl.Table(Data_array, names = (Col_names))

# Show table in notebook
t.show_in_notebook()

#Save table to LaTex format (uncomment if saving)
#t.write("Table_LaTex.txt", format='latex')
#t.write("Table.txt", format = 'ascii.tab')

idx,Galaxy,Halo Mass $(10^{12}M_{\odot})$,Disk Mass $(10^{12}M_{\odot})$,Bulge Mass $(10^{12}M_{\odot})$,Total Mass $(10^{12}M_{\odot})$,$f_{bar}$
0,MW,1.975,0.075,0.01,2.06,0.041
1,M31,1.921,0.12,0.019,2.06,0.067
2,M33,0.187,0.009,0.0,0.196,0.046
3,Local Group,4.083,0.204,0.029,4.316,0.054
